In [16]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import sklearn

import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import time 

from utils.utils import create_directory

In [17]:
root_dir="/data4/gsPrivate/dl-4-tsc"

archive_name="UCRArchive_2018"
dataset_name="Coffee"
datasets_dict = {}
cur_root_dir = root_dir.replace('-temp', '')


root_dir_dataset = cur_root_dir + '/archives/' + archive_name + '/' + dataset_name + '/'
df_train = pd.read_csv(root_dir_dataset + '/' + dataset_name + '_TRAIN.tsv', sep='\t', header=None)

df_test = pd.read_csv(root_dir_dataset + '/' + dataset_name + '_TEST.tsv', sep='\t', header=None)


In [18]:

y_train = df_train.values[:, 0]
y_test = df_test.values[:, 0]

x_train = df_train.drop(columns=[0])
x_test = df_test.drop(columns=[0])


In [19]:
x_train.shape,y_train.shape

((28, 286), (28,))

In [20]:

# x_train.columns = range(x_train.shape[1])
# x_test.columns = range(x_test.shape[1])


In [21]:

x_train = x_train.values
x_test = x_test.values


In [22]:
x_train

array([[-0.51841899, -0.48588363, -0.50500747, ..., -1.9336308 ,
        -1.9349635 , -1.9360067 ],
       [-0.54846169, -0.53368082, -0.51472295, ..., -1.9407273 ,
        -1.9437857 , -1.9443082 ],
       [-0.47263392, -0.41554604, -0.3599294 , ..., -1.9891427 ,
        -1.9906587 , -1.9923005 ],
       ...,
       [-0.53874235, -0.47511614, -0.48579124, ..., -1.8360855 ,
        -1.837943  , -1.839142  ],
       [-0.59182708, -0.57097984, -0.59002393, ..., -1.8736945 ,
        -1.8752949 , -1.8771268 ],
       [-0.69637574, -0.64382723, -0.65514167, ..., -1.7773633 ,
        -1.7789851 , -1.7804869 ]])

In [23]:



# znorm
std_ = x_train.std(axis=1, keepdims=True)
std_[std_ == 0] = 1.0
x_train = (x_train - x_train.mean(axis=1, keepdims=True)) / std_

std_ = x_test.std(axis=1, keepdims=True)
std_[std_ == 0] = 1.0
x_test = (x_test - x_test.mean(axis=1, keepdims=True)) / std_

datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                y_test.copy())



In [24]:
x_train.shape,y_train.shape

((28, 286), (28,))

In [25]:
np.concatenate((y_train, y_test), axis=0),y_test

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

# fit classifier

In [26]:

nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

# transform the labels from integers to one hot vectors
enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

# save orignal y because later we will use binary
y_true = np.argmax(y_test, axis=1)

if len(x_train.shape) == 2:  # if univariate
    # add a dimension to make it multivariate with one dimension 
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

input_shape = x_train.shape[1:]

In [29]:
y_train.shape,x_train.shape,x_test.shape,y_test.shape,y_test,y_true

((28, 2),
 (28, 286, 1),
 (28, 286, 1),
 (28, 2),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1]))

In [ ]:
input_shape,nb_classes

((286, 1), 2)

In [ ]:

print("TensorFlow version:", tf.__version__)

# 检查可用的 GPU 设备
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print("GPU Available:", gpu)
else:
    print("No GPU Available.")

TensorFlow version: 2.13.1
No GPU Available.


2023-12-20 07:25:56.623938: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2023-12-20 07:25:56.624020: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: soft
2023-12-20 07:25:56.624033: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: soft
2023-12-20 07:25:56.624164: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 510.108.3
2023-12-20 07:25:56.624203: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 510.47.3
2023-12-20 07:25:56.624213: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 510.47.3 does not match DSO version 510.108.3 -- cannot find working devices in this configuration


In [ ]:

def create_classifier(classifier_name, input_shape, nb_classes, output_directory, verbose=True):
    if classifier_name == 'fcn':
        from classifiers import fcn
        return fcn.Classifier_FCN(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'mlp':
        from classifiers import mlp
        return mlp.Classifier_MLP(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'resnet':
        from classifiers import resnet
        return resnet.Classifier_RESNET(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'mcnn':
        from classifiers import mcnn
        return mcnn.Classifier_MCNN(output_directory, verbose)
    if classifier_name == 'tlenet':
        from classifiers import tlenet
        return tlenet.Classifier_TLENET(output_directory, verbose)
    if classifier_name == 'twiesn':
        from classifiers import twiesn
        return twiesn.Classifier_TWIESN(output_directory, verbose)
    if classifier_name == 'encoder':
        from classifiers import encoder
        return encoder.Classifier_ENCODER(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'mcdcnn':
        from classifiers import mcdcnn
        return mcdcnn.Classifier_MCDCNN(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'cnn':  # Time-CNN
        from classifiers import cnn
        return cnn.Classifier_CNN(output_directory, input_shape, nb_classes, verbose)
    if classifier_name == 'inception':
        from classifiers import inception
        return inception.Classifier_INCEPTION(output_directory, input_shape, nb_classes, verbose)



In [ ]:

classifier_name = "fcn"

output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '' + '/' + \
                    dataset_name + '/'
create_directory(output_directory)

In [ ]:
classifier = create_classifier(classifier_name, input_shape, nb_classes, output_directory)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 286, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 286, 128)          1152      
                                                                 
 batch_normalization (Batch  (None, 286, 128)          512       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 286, 128)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 286, 256)          164096    
                                                                 
 batch_normalization_1 (Bat  (None, 286, 256)          1024      
 chNormalization)                                            

In [ ]:
classifier.fit(x_train, y_train, x_test, y_test, y_true,300)

Epoch 1/100
14/14 [==============================] - 3s 53ms/step - loss: 0.7211 - accuracy: 0.4643 - val_loss: 0.6899 - val_accuracy: 0.5714 - lr: 0.0010
Epoch 2/100
 7/14 [==============>...............] - ETA: 0s - loss: 0.7286 - accuracy: 0.4286

/data4/conda_envs/gsprivate/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


14/14 [==============================] - 0s 29ms/step - loss: 0.7106 - accuracy: 0.5000 - val_loss: 0.6879 - val_accuracy: 0.5357 - lr: 0.0010
Epoch 3/100
14/14 [==============================] - 0s 29ms/step - loss: 0.6573 - accuracy: 0.8214 - val_loss: 0.6920 - val_accuracy: 0.5357 - lr: 0.0010
Epoch 4/100
14/14 [==============================] - 0s 25ms/step - loss: 0.6685 - accuracy: 0.6071 - val_loss: 0.6914 - val_accuracy: 0.5357 - lr: 0.0010
Epoch 5/100
14/14 [==============================] - 0s 30ms/step - loss: 0.6396 - accuracy: 0.5714 - val_loss: 0.6916 - val_accuracy: 0.5357 - lr: 0.0010
Epoch 6/100
14/14 [==============================] - 0s 29ms/step - loss: 0.6395 - accuracy: 0.6786 - val_loss: 0.7172 - val_accuracy: 0.5357 - lr: 0.0010
Epoch 7/100
14/14 [==============================] - 0s 31ms/step - loss: 0.5649 - accuracy: 0.8571 - val_loss: 0.7368 - val_accuracy: 0.5357 - lr: 0.0010
Epoch 8/100
14/14 [==============================] - 0s 27ms/step - loss: 0.6245 -

findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because

In [20]:
import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2.13.1
Num GPUs Available:  0
